In [30]:
import os

In [31]:
os.environ.setdefault('AWS_PROFILE', 'ghactivity')

'ghactivity'

In [32]:
!aws s3 ls s3://aigithub/landing/ghactivity/

2022-06-08 21:24:04   33989636 2022-06-05-0.json.gz
2022-06-08 21:44:13   39078924 2022-06-05-1.json.gz
2022-06-14 19:10:52   45518853 2022-06-05-10.json.gz
2022-06-14 19:16:54   40533549 2022-06-05-11.json.gz
2022-06-13 10:21:47   34733178 2022-06-05-2.json.gz
2022-06-13 17:09:33   36298891 2022-06-05-3.json.gz
2022-06-13 18:00:14   28220189 2022-06-05-4.json.gz
2022-06-13 18:01:13   30436488 2022-06-05-5.json.gz
2022-06-13 18:10:06   31640337 2022-06-05-6.json.gz
2022-06-13 18:33:41   33084474 2022-06-05-7.json.gz
2022-06-14 09:49:19   41575819 2022-06-05-8.json.gz
2022-06-14 17:47:37   38022512 2022-06-05-9.json.gz


In [4]:
!aws s3 rm s3://aigithub/raw/ghactivity/ --recursive

delete: s3://aigithub/raw/ghactivity/year=2022/month=06/dayofmonth=05/part-2022-06-05-0-bed449c4-edf5-11ec-b327-e69f877c2171.snappy.parquet
delete: s3://aigithub/raw/ghactivity/year=2022/month=06/dayofmonth=05/part-2022-06-05-0-66907c10-edf5-11ec-b327-e69f877c2171.snappy.parquet
delete: s3://aigithub/raw/ghactivity/year=2022/month=06/dayofmonth=05/part-2022-06-05-0-a5549936-edf5-11ec-b327-e69f877c2171.snappy.parquet
delete: s3://aigithub/raw/ghactivity/year=2022/month=06/dayofmonth=05/part-2022-06-05-0-8acac0f4-edf5-11ec-b327-e69f877c2171.snappy.parquet
delete: s3://aigithub/raw/ghactivity/year=2022/month=06/dayofmonth=05/part-2022-06-05-0-b244c292-edf5-11ec-b327-e69f877c2171.snappy.parquet
delete: s3://aigithub/raw/ghactivity/year=2022/month=06/dayofmonth=05/part-2022-06-05-0-7b27f68a-edf5-11ec-b327-e69f877c2171.snappy.parquet
delete: s3://aigithub/raw/ghactivity/year=2022/month=06/dayofmonth=05/part-2022-06-05-0-91a055f6-edf5-11ec-b327-e69f877c2171.snappy.parquet
delete: s3://aigithu

In [5]:
import pandas as pd

In [6]:
bucket_name = 'aigithub'
file_name = '2022-06-05-0.json.gz'

In [7]:
df = pd.read_json(
    f's3://{bucket_name}/landing/ghactivity/{file_name}',
    lines=True,
    orient='records'
)

In [8]:
type(df)

pandas.core.frame.DataFrame

In [9]:
df.columns

Index(['id', 'type', 'actor', 'repo', 'payload', 'public', 'created_at',
       'org'],
      dtype='object')

In [10]:
year = file_name.split('-')[0]
month = file_name.split('-')[1]
dayofmonth = file_name.split('-')[2]

In [13]:
import uuid

target_file_name = f'{uuid.uuid1()}.snappy.parquet'

In [14]:
target_file_name

'd55db3a6-ee1d-11ec-b1c2-e69f877c2170.snappy.parquet'

In [15]:
df.drop?

Signature:
df.drop(
    labels=None,
    axis: 'Axis' = 0,
    index=None,
    columns=None,
    level: 'Level | None' = None,
    inplace: 'bool' = False,
    errors: 'str' = 'raise',
)
Docstring:
Drop specified labels from rows or columns.

Remove rows or columns by specifying label names and corresponding
axis, or by specifying directly index or column names. When using a
multi-index, labels on different levels can be removed by specifying
the level. See the `user guide <advanced.shown_levels>`
for more information about the now unused levels.

Parameters
----------
labels : single label or list-like
    Index or column labels to drop. A tuple will be used as a single
    label and not treated as a list-like.
axis : {0 or 'index', 1 or 'columns'}, default 0
    Whether to drop labels from the index (0 or 'index') or
    columns (1 or 'columns').
index : single label or list-like
    Alternative to specifying axis (``labels, axis=0``
    is equivalent to ``index=labels``).
columns : 

In [16]:
df.to_parquet?

Signature:
df.to_parquet(
    path: 'FilePath | WriteBuffer[bytes] | None' = None,
    engine: 'str' = 'auto',
    compression: 'str | None' = 'snappy',
    index: 'bool | None' = None,
    partition_cols: 'list[str] | None' = None,
    storage_options: 'StorageOptions' = None,
    **kwargs,
) -> 'bytes | None'
Docstring:
Write a DataFrame to the binary parquet format.

This function writes the dataframe as a `parquet file
<https://parquet.apache.org/>`_. You can choose different parquet
backends, and have the option of compression. See
:ref:`the user guide <io.parquet>` for more details.

Parameters
----------
path : str, path object, file-like object, or None, default None
    String, path object (implementing ``os.PathLike[str]``), or file-like
    object implementing a binary ``write()`` function. If None, the result is
    returned as bytes. If a string or path, it will be used as Root Directory
    path when writing a partitioned dataset.

    .. versionchanged:: 1.2.0

    Previ

In [17]:
# This lift and shift model might run into out of memory issues
# as the operation is both memory and CPU Intensive
# I have tested this as lambda and it is failing even with 10 GB memory (which is max)
df.drop(columns=['payload']). \
    to_parquet(
        f's3://{bucket_name}/raw/ghactivity/year={year}/month={month}/dayofmonth={dayofmonth}/{target_file_name}',
        index=False
    )

In [18]:
!aws s3 ls s3://aigithub/raw/ghactivity --recursive

2022-06-17 14:48:10    5655036 raw/ghactivity/year=2022/month=06/dayofmonth=05/d55db3a6-ee1d-11ec-b1c2-e69f877c2170.snappy.parquet


In [ ]:
# Instead of lift and shift, let us take care of uploading the data in chunks
# Every 10K records will be converted and uploaded using parquet file format

In [19]:
# Deleting all files which is already uploaded
!aws s3 rm s3://aigithub/raw/ghactivity/ --recursive

delete: s3://aigithub/raw/ghactivity/year=2022/month=06/dayofmonth=05/d55db3a6-ee1d-11ec-b1c2-e69f877c2170.snappy.parquet


In [20]:
bucket_name = 'aigithub'
file_name = '2022-06-05-0.json.gz'

In [21]:
# Read the data from json file in chunks
# Reader type object will be created rather than Dataframe

df_reader = pd.read_json(
    f's3://{bucket_name}/landing/ghactivity/{file_name}',
    lines=True,
    orient='records',
    chunksize=10000
)

In [22]:
type(df_reader)

pandas.io.json._json.JsonReader

In [23]:
# We can iterate through reader in streaming fashion using for loop
# When you validate from your local machine, this will take time based on the internet speed
for idx, df in enumerate(df_reader):
    print(f'The number of records in chunk {idx}: {df.shape[0]}')

The number of records in chunk 0: 10000
The number of records in chunk 1: 10000
The number of records in chunk 2: 10000
The number of records in chunk 3: 10000
The number of records in chunk 4: 10000
The number of records in chunk 5: 10000
The number of records in chunk 6: 10000
The number of records in chunk 7: 10000
The number of records in chunk 8: 9443


In [24]:
year = file_name.split('-')[0]
month = file_name.split('-')[1]
dayofmonth = file_name.split('-')[2]
hour = file_name.split('-')[3].split('.')[0]

In [25]:
target_file_name = f'part-{year}-{month}-{dayofmonth}-{hour}-{uuid.uuid1()}.snappy.parquet'
target_file_name

'part-2022-06-05-0-b3020a16-ee20-11ec-b1c2-e69f877c2170.snappy.parquet'

In [26]:
df.to_parquet?

Signature:
df.to_parquet(
    path: 'FilePath | WriteBuffer[bytes] | None' = None,
    engine: 'str' = 'auto',
    compression: 'str | None' = 'snappy',
    index: 'bool | None' = None,
    partition_cols: 'list[str] | None' = None,
    storage_options: 'StorageOptions' = None,
    **kwargs,
) -> 'bytes | None'
Docstring:
Write a DataFrame to the binary parquet format.

This function writes the dataframe as a `parquet file
<https://parquet.apache.org/>`_. You can choose different parquet
backends, and have the option of compression. See
:ref:`the user guide <io.parquet>` for more details.

Parameters
----------
path : str, path object, file-like object, or None, default None
    String, path object (implementing ``os.PathLike[str]``), or file-like
    object implementing a binary ``write()`` function. If None, the result is
    returned as bytes. If a string or path, it will be used as Root Directory
    path when writing a partitioned dataset.

    .. versionchanged:: 1.2.0

    Previ

In [27]:
# As df_reader is already processed, if we try to process again it will fail
# Make sure to create reader object again to process while writing to s3
df_reader = pd.read_json(
    f's3://{bucket_name}/landing/ghactivity/{file_name}',
    lines=True,
    orient='records',
    chunksize=10000
)

In [28]:
for idx, df in enumerate(df_reader):
    target_file_name = f'part-{year}-{month}-{dayofmonth}-{hour}-{uuid.uuid1()}.snappy.parquet'
    print(f'Processing chunk {idx} of size {df.shape[0]} from {file_name}')
    df.drop(columns=['payload']). \
    to_parquet(
        f's3://{bucket_name}/raw/ghactivity/year={year}/month={month}/dayofmonth={dayofmonth}/{target_file_name}',
        index=False
    )

Processing chunk 0 of size 10000 from 2022-06-05-0.json.gz
Processing chunk 1 of size 10000 from 2022-06-05-0.json.gz
Processing chunk 2 of size 10000 from 2022-06-05-0.json.gz
Processing chunk 3 of size 10000 from 2022-06-05-0.json.gz
Processing chunk 4 of size 10000 from 2022-06-05-0.json.gz
Processing chunk 5 of size 10000 from 2022-06-05-0.json.gz
Processing chunk 6 of size 10000 from 2022-06-05-0.json.gz
Processing chunk 7 of size 10000 from 2022-06-05-0.json.gz
Processing chunk 8 of size 9443 from 2022-06-05-0.json.gz


In [29]:
!aws s3 ls s3://aigithub/raw/ghactivity/ --recursive

2022-06-17 15:08:37     735024 raw/ghactivity/year=2022/month=06/dayofmonth=05/part-2022-06-05-0-425d2830-ee21-11ec-b1c2-e69f877c2170.snappy.parquet
2022-06-17 15:09:57     717761 raw/ghactivity/year=2022/month=06/dayofmonth=05/part-2022-06-05-0-726e6c50-ee21-11ec-b1c2-e69f877c2170.snappy.parquet
2022-06-17 15:10:28     690393 raw/ghactivity/year=2022/month=06/dayofmonth=05/part-2022-06-05-0-84bccb86-ee21-11ec-b1c2-e69f877c2170.snappy.parquet
2022-06-17 15:11:24     718011 raw/ghactivity/year=2022/month=06/dayofmonth=05/part-2022-06-05-0-a64fea9e-ee21-11ec-b1c2-e69f877c2170.snappy.parquet
2022-06-17 15:11:47     728950 raw/ghactivity/year=2022/month=06/dayofmonth=05/part-2022-06-05-0-b3f9908c-ee21-11ec-b1c2-e69f877c2170.snappy.parquet
2022-06-17 15:12:22     753367 raw/ghactivity/year=2022/month=06/dayofmonth=05/part-2022-06-05-0-c8d1227c-ee21-11ec-b1c2-e69f877c2170.snappy.parquet
2022-06-17 15:13:11     753795 raw/ghactivity/year=2022/month=06/dayofmonth=05/part-2022-06-05-0-e5f4e9c4-